In [25]:
import pandas as pd

In [26]:
# Load dataset
df_titanic = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

In [27]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)

    df['Embarked'].fillna('S', inplace=True)

    return  df

df_titanic = handle_null_median(df_titanic)

In [28]:
df_titanic['Sex'] = df_titanic['Sex'].replace(['female', 'male'], [0,1])

In [29]:
df_titanic['FamilyMembersCount'] = df_titanic['Parch'] + df_titanic['SibSp'] + 1
df_titanic = df_titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

In [30]:
df_XGB = df_titanic.copy()
df_RF = df_titanic.copy()
df_GNB = df_titanic.copy()

In [31]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    # Could also create bins for fare, but not sure Fare is needed.
    return df

df_XGB = bins(df_titanic)
df_XGB

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilyMembersCount,AgeGroup
0,0,3,1,22.0,7.2500,S,2,1
1,1,1,0,38.0,71.2833,C,2,2
2,1,3,0,26.0,7.9250,S,1,1
3,1,1,0,35.0,53.1000,S,2,2
4,0,3,1,35.0,8.0500,S,1,2
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,S,1,1
887,1,1,0,19.0,30.0000,S,1,1
888,0,3,0,28.0,23.4500,S,4,1
889,1,1,1,26.0,30.0000,C,1,1


In [13]:
df_titanic['Pclass'] =  df_titanic['Pclass'].astype('category')
df_titanic['Embarked'] =  df_titanic['Embarked'].astype('category')
df_titanic['AgeGroup'] =  df_titanic['AgeGroup'].astype('category')

In [14]:
df_XGB = pd.get_dummies(df_titanic)
df_RF = pd.get_dummies(df_titanic)
df_GNB = pd.get_dummies(df_titanic)

In [15]:
from sklearn.model_selection import train_test_split

# XGBoost

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

train_X, test_X, train_y, test_y = train_test_split(df_XGB.drop('Survived', axis=1), df_XGB['Survived'], random_state=0)

model = XGBClassifier()
model.fit(train_X, train_y)

pred_y = model.predict(test_X)
preds = [round(value) for value in pred_y]

# evaluate predictions
accuracy = accuracy_score(test_y, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[21:32:42] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 84.30%


C:\Users\marku\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [17]:
from sklearn.model_selection import GridSearchCV
XGBParam = {'eta': [0.01, 0.05, 0.1, 0.15, 0.2],
            'gamma': [0, 1, 10, 15],
            'max_depth': [3, 5, 6, 8, 10],
            'subsample': [0.5, 0.75, 1]
            }

XGB_grid = GridSearchCV(XGBClassifier(verbosity=0, tree_method='auto'), XGBParam)
XGB_grid.fit(train_X, train_y)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
            

In [18]:
print(XGB_grid.best_score_)

0.8323083828975424


In [19]:
print(XGB_grid.best_params_)

{'eta': 0.1, 'gamma': 0, 'max_depth': 5, 'subsample': 0.5}


# Random Forrest Classifier

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('Survived', axis=1), df_RF['Survived'], random_state=0)

# Train and test random forest classifier with default parameters
rf_scores = cross_val_score(RandomForestClassifier(), train_X, train_y)
(rf_scores.mean(), rf_scores.std())

(0.7949051733812141, 0.03165187848771408)

In [21]:
RFParam = {"max_features":['auto'],
            "max_depth":[2, 6, 12, 24, 48, 96],
            "n_estimators":[50, 100, 200, 300]}

RF_Grid = GridSearchCV(RandomForestClassifier(), RFParam)
RF_Grid.fit(train_X, train_y)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 6, 12, 24, 48, 96],
                         'max_features': ['auto'],
                         'n_estimators': [50, 100, 200, 300]})

In [22]:
print(RF_Grid.best_score_)

0.8203344181348895


In [23]:
print(RF_Grid.best_params_)

{'max_depth': 6, 'max_features': 'auto', 'n_estimators': 300}


# GaussianNB

In [24]:
from sklearn.naive_bayes import GaussianNB

train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('Survived', axis=1), df_RF['Survived'], random_state=0)

# Train and test random forest classifier with default parameters
GNB_scores = cross_val_score(GaussianNB(), train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

(0.7396588486140725, 0.06916285283869081)